In [1]:
import os
import glob
import pandas as pd

import geopandas as gpd
import ipyleaflet as ipl

import sys
from pathlib import Path
sys.path.insert(0,str(Path(os.path.abspath('.')).parent.parent))
import library.validation.Interactive_map.continent_and_country as continent_and_country
import library.validation.Interactive_map.mapping as mapping

# metadata for all stations

In [2]:
common_path = "/home/lhn3e/OneDrive-3E/Research/Solar/tickets/2023/IN*_measurement_data/metadata/data.csv"
file_paths = glob.glob(common_path)

In [3]:
dfs = []
for file in file_paths:
    station_name = os.path.basename(os.path.dirname(os.path.dirname(file))).split('_')[1]
    df = pd.read_csv(file, index_col=0)
    df['provider'] = station_name 
    dfs.append(df)

In [4]:
result = pd.concat(dfs)


In [5]:
nan_name = result.index[result.index.isna()]
for i, nn in enumerate(nan_name):
    result = result.rename(index={nn:f'Unnamed Station {i + 1}'})

In [5]:
result

,latitude,longitude,elevation (m),provider,country,time zone,time resolution
Bondville_Illinois,40.050000,-88.370000,213.0,SURFRAD,NaN,NaN,NaN
Desert_Rock_Nevada,36.624000,-116.019000,1007.0,SURFRAD,NaN,NaN,NaN
Fort_Peck_Montana,48.310000,-105.100000,634.0,SURFRAD,NaN,NaN,NaN
Goodwin_Creek_Mississippi,34.250000,-89.870000,98.0,SURFRAD,NaN,NaN,NaN
Penn_State_Pennsylvania,40.720000,-77.930000,376.0,SURFRAD,NaN,NaN,NaN
...,...,...,...,...,...,...,...
MENE-unnamed station 11,39.298833,-76.587833,NaN,enerMENA,NaN,NaN,NaN
MENE-unnamed station 12,39.906020,-75.181032,NaN,enerMENA,NaN,NaN,NaN
MENE-unnamed station 13,39.936557,-75.173743,NaN,enerMENA,NaN,NaN,NaN
MENE-unnamed station 14,42.363672,-71.101502,NaN,enerMENA,NaN,NaN,NaN


# country and continent

In [6]:
continents_and_countries = continent_and_country.get_continents_and_countries()
result['coordinates'] = result[['latitude', 'longitude']].apply(lambda x: f"{x['latitude']},{x['longitude']}", axis=1)
result[['coordinates', 'country', 'continent']] = result['coordinates'].apply(lambda coo: pd.Series(continent_and_country.get_location_of(coo, continents_and_countries)))


In [8]:
result.to_csv('data.csv', index_label = 'station name')

# geodataframe

In [9]:
result_gdf = gpd.GeoDataFrame(result, geometry=gpd.points_from_xy(result.longitude, result.latitude))

# map

In [10]:
markers=[
    mapping._convert_site_to_marker_with_popup(gdf=result_gdf, station=station) for station in result_gdf.index
]

In [11]:
map = mapping.IPLBaseMap()

In [12]:
markers = ipl.MarkerCluster(markers=markers,
                  disable_clustering_at_zoom=3,
                  max_cluster_radius=100,
                           name="validation sites")

In [13]:
map.add_layer(markers)
map

IPLBaseMap(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [14]:
map.save('/home/lhn3e/OneDrive-3E/Research/Solar/tickets/2023/IN2887_interactive_map/interactive map for measurement data.html')
